In [191]:
from ucimlrepo import fetch_ucirepo
from random import randint
from random import random
from random import shuffle
from random import sample
import numpy
import math

In [2]:
iris = fetch_ucirepo(id=53) 

In [200]:
glass = fetch_ucirepo(id=42)

In [384]:
def toNum(dataset):
    x = dataset.data.features.to_numpy()
    y = dataset.data.targets.to_numpy()
    return x,numpy.ravel(y)

In [12]:
def getClases(target):
    # Obtiene los diferentes clases
    s = set()
    for a in target:
        s.add(a)
    return list(s)

In [13]:
getClases(toNum(iris)[1])

['Iris-versicolor', 'Iris-setosa', 'Iris-virginica']

In [94]:
def obtenerValores(pares, e):
    tp = 0 # true negative
    fn = 0 # false negative
    fp = 0 # false positive
    tn = 0 # true negative
    for a in pares:
        if a[0]==e:
            # Realmente es positivo
            if a[1]==e:
                tp+=1
            else:
                fn+=1
        else:
             if a[1]==e:
                 fp+=1
             else:
                 tn+=1
    return [tp,fn,fp,tn]

def obtenerAccuracy(pares,e):
    tp,fn,fp,tn=obtenerValores(pares,e)
    return (tp+tn)/(tp+tn+fp+fn)

def matEsp(pares, e): # Realiza el matriz y el accuracy sobre un solo valor
    tp,fn,fp,tn=obtenerValores(pares,e)
    print(f"Elaborado sobre {e}")
    print(f"Accuracy: {(tp+tn)/(tp+tn+fp+fn)}")
    print("\t1\t0")
    print(f"1\t{tp}\t{fn}")
    print(f"0\t{fp}\t{tn}")

def obtenerAccGlobal(pares):
    s = set()
    for a in pares:
        s.add(a[0])
    prom = []
    for a in s:
        prom.append(obtenerAccuracy(pares,a))
    return sum(prom)/len(prom)

    
def accYMat(pares): # Sobre los pares de (real,prediccion), se obtiene el accuracy y el matriz de confusion
    s = set()
    for a in pares:
        s.add(a[0])
    print("Arriba es la clase predicha, a la izquierda es la clase real.")
    prom = []
    for a in s:
        prom.append(obtenerAccuracy(pares,a))
    print(f"Accuracy global: {sum(prom)/len(prom)}")
    print("<---")
    for a in s:
        matEsp(pares,a)
    print("--->")

In [35]:
def dist(a, b):
    # Retorna la distancia entre a y b
    if len(a) == len(b):
        acum = 0
        for x in range(len(a)):
            acum += (a[x] - b[x])**2
        return math.sqrt(acum)
    else:
        return -1

In [216]:
class unoNN:
    def fit(this,matEnt, valores):
        this.matEnt = matEnt
        this.valores = valores
    def pred(this,dato):
        distMin = dist(dato,this.matEnt[0])
        posMin = 0
        for a in range(1,len(this.matEnt)):
            d = dist(dato,this.matEnt[a])
            if(d < distMin):
                posMin = a
                distMin = d
        return this.valores[posMin]
    def predict(this,datos):
        ret = []
        for a in datos:
            ret.append(this.pred(a))
        return ret

In [122]:
def separar(matEnt,valores):
    # Separa la matriz de entrada en indices correspondientes a cada clase clave
    separar = {}
    clases = getClases(valores)
    for a in clases:
        separar[a] = []
    # Este hold out no sera aleatorio
    for a in range(len(matEnt)):
        # separar[valores[a]].append(matEnt[a])
        separar[valores[a]].append(a)
    return separar

def holdOut(matEnt, valores, r=0.7):
    # Separamos los valores de acuerdo a sus clases
    sep = separar(matEnt,valores)
    ent,prueba = [],[]
    for a in sep.keys():
        entTam = int(len(sep[a])*r)
        ent += sep[a][:entTam]
        prueba += sep[a][entTam:]
    res = [[ent,prueba]]
    return res

In [364]:
def kFold(matEnt,valores,k=10):
    nums = list(range(len(matEnt)))
    shuffle(nums)
    divTam = int(len(matEnt)/k)
    print(len(matEnt))
    folds = []
    for a in range(k):
        folds.append(nums[a*divTam:(a+1)*divTam])
    ret = []
    for a in range(k):
        merge = []
        for b in range(k):
            if a!=b:
                merge += folds[b]
        ret.append([merge, folds[a]])
    return ret

In [214]:
def correr(datos,algoritmo,metodoVal):
    real = numpy.array([])
    pred = numpy.array([])
    for indEnt,indVal in metodoVal(datos[0],datos[1]):
        x = algoritmo()
        x.fit(datos[0][indEnt], datos[1][indEnt])
        real = numpy.concatenate([real, datos[1][indVal]])
        pred = numpy.concatenate([pred, numpy.array(x.predict(datos[0][indVal]))])
    return list(zip(list(real),list(pred)))

In [180]:
def obtenerVecinos(matEnt, dato, k=5):
    # Obtiene los k puntos mas cercanos de dato (de lo que se encuentra en datos) (como indice)
    ret = []
    for a in range(len(matEnt)):
        ret.append((a,dist(matEnt[a], dato)))
    ret.sort(key=lambda x: x[1])
    if(ret[0][1] == 0): # Si el elemento mas cercano es el mismo dato, lo excluimos
        ret = ret[1:k+1]
    else:
        ret = ret[0:k];
    return [x[0] for x in ret]

In [168]:
def puntoEntre(a,b):
    # Retorna un punto aleatorio entre dos vectores, excluyendo a,b
    diff = a-b
    return b + diff*random()

In [501]:
def smote(datos, claseMin, N=3, k=5):
    # De los K vecinos mas cercanos, selecciona N de forma aleatoria. Requisitos k>len(datosMin) y k>N
    # Retorna len(claseMin)*N Nuevos datos
    sep = separar(datos[0],datos[1]) # Obtenemos los indices para cada clase
    indMin = sep[claseMin]
    datosEntMin = datos[0][indMin]
    if(len(indMin) < k):
        k = len(indMin)
        print("K muy grande para la clase minima")
    # datosValMin = datos[1][indMin] # Dato irrelevante al ser todas de la misma clase
    ret = []
    for a in datosEntMin:
        vecinos = obtenerVecinos(datosEntMin,a,k)
        nInd = sample(vecinos,N)
        n = datosEntMin[nInd]
        for b in n:
            ret.append(puntoEntre(a,b))
    return ret
def smote2(datosEntMin):
    # Version simplificada de smote,toma un punto, determina k=3, y N=1 por una sola iteracion
    p = sample(range(len(datosEntMin)), 1)
    punto = datosEntMin[p[0]]
    vecs = datosEntMin[obtenerVecinos(datosEntMin, punto, k=3)]
    p2 = vecs[randint(0,2)]
    return puntoEntre(p,p2)
    #ret = puntoEntre(datosEntMin[puntos[0]],datosEntMin[puntos[0]])
    #return ret

In [205]:
glassNum = toNum(glass)

In [366]:
def agregarDatos(datos):
    sep = separar(*datos)
    tams = {}
    m = list(sep.keys())[0]
    for a in sep.keys():
        tams[a] = len(sep[a])
        if(tams[a] > tams[m]):
            m = a
    diffMin = 10 # Si la diferencia entre dos clases, es menor a la minima de diferencia, se ignora la clase
    ret = []
    val = []
    for a in sep.keys():
        diff = tams[m] - tams[a]
        ret += list(datos[0][sep[a]])
        val += list(datos[1][sep[a]])
        if(diff >= diffMin):
            dats = datos[0][sep[a]]
            for b in range(diff):
                ret.append(smote2(dats))
                val.append(a)

    return numpy.asarray([ret,val],dtype="object")

In [506]:
res = correr(glassNum,unoNN,kFold)
res2 = correr(agregarDatos(glassNum),unoNN,kFold)
accYMat(res)
accYMat(res2)

214
450
Arriba es la clase predicha, a la izquierda es la clase real.
Accuracy global: 0.9126984126984127
<---
Elaborado sobre 1.0
Accuracy: 0.819047619047619
	1	0
1	53	15
0	23	119
Elaborado sobre 2.0
Accuracy: 0.8523809523809524
	1	0
1	56	19
0	12	123
Elaborado sobre 3.0
Accuracy: 0.9047619047619048
	1	0
1	6	11
0	9	184
Elaborado sobre 5.0
Accuracy: 0.9714285714285714
	1	0
1	10	2
0	4	194
Elaborado sobre 6.0
Accuracy: 0.9666666666666667
	1	0
1	5	4
0	3	198
Elaborado sobre 7.0
Accuracy: 0.9619047619047619
	1	0
1	25	4
0	4	177
--->
Arriba es la clase predicha, a la izquierda es la clase real.
Accuracy global: 0.8829629629629631
<---
Elaborado sobre 1
Accuracy: 0.9088888888888889
	1	0
1	55	15
0	26	354
Elaborado sobre 2
Accuracy: 0.92
	1	0
1	54	22
0	14	360
Elaborado sobre 3
Accuracy: 0.8488888888888889
	1	0
1	43	33
0	35	339
Elaborado sobre 5
Accuracy: 0.8822222222222222
	1	0
1	47	29
0	24	350
Elaborado sobre 6
Accuracy: 0.8466666666666667
	1	0
1	45	31
0	38	336
Elaborado sobre 7
Accuracy: 0.8911

In [507]:
res = correr(glassNum,unoNN,holdOut)
res2 = correr(agregarDatos(glassNum),unoNN,holdOut)
accYMat(res)
accYMat(res2)

Arriba es la clase predicha, a la izquierda es la clase real.
Accuracy global: 0.8535353535353535
<---
Elaborado sobre 1.0
Accuracy: 0.6666666666666666
	1	0
1	13	8
0	14	31
Elaborado sobre 2.0
Accuracy: 0.6818181818181818
	1	0
1	9	14
0	7	36
Elaborado sobre 3.0
Accuracy: 0.8787878787878788
	1	0
1	1	5
0	3	57
Elaborado sobre 5.0
Accuracy: 0.9696969696969697
	1	0
1	4	0
0	2	60
Elaborado sobre 6.0
Accuracy: 0.9242424242424242
	1	0
1	1	2
0	3	60
Elaborado sobre 7.0
Accuracy: 1.0
	1	0
1	9	0
0	0	57
--->
Arriba es la clase predicha, a la izquierda es la clase real.
Accuracy global: 0.8455882352941176
<---
Elaborado sobre 1
Accuracy: 0.875
	1	0
1	13	8
0	9	106
Elaborado sobre 2
Accuracy: 0.8529411764705882
	1	0
1	9	14
0	6	107
Elaborado sobre 3
Accuracy: 0.7867647058823529
	1	0
1	13	10
0	19	94
Elaborado sobre 5
Accuracy: 0.8602941176470589
	1	0
1	15	8
0	11	102
Elaborado sobre 6
Accuracy: 0.8382352941176471
	1	0
1	14	9
0	13	100
Elaborado sobre 7
Accuracy: 0.8602941176470589
	1	0
1	9	14
0	5	108
--->


## Parte 2

In [385]:
irisNum = toNum(iris)

In [388]:
irisClases = getClases(irisNum[1])
irisClases

['Iris-versicolor', 'Iris-setosa', 'Iris-virginica']

In [389]:
filtro = [irisClases[0] != x for x in irisNum[1]]
iris2 = [irisNum[0][filtro],irisNum[1][filtro]]

In [465]:
class perceptron():
    # Se implementa la regla de hebb para el aprendizaje
    ritmo = 0.5 # Ritmo de aprendizaje
    def fit(this,matEnt, sal):
        this.clases = getClases(sal) # Primer elemento sera -1, y el segundo elemento sera 1
        t = [-1 if x == this.clases[0] else 1 for x in sal]
        this.pesos = [0]*len(matEnt[0]) # La cantidad de columnas del primer elemento
        for a,b in zip(matEnt,t):
            this.pesos = this.pesos + this.ritmo*a*b
        activ1,activ2 = [],[]
        for a,b in zip(matEnt,t):
            suma = 0
            for c in range(len(this.pesos)):
                suma += this.pesos[c] * a[c]
            if(b == 1):
                activ2.append(suma)
            else:
                activ1.append(suma)
        if(min(activ1) > max(activ2)):
            ag = activ1
            am = activ2
            this.mg = 0
        else:
            ag = activ2
            am = activ1
            this.mg = 1
        this.thresh = (min(ag) + max(am))/2
    def pred(this,dato):
        suma = 0
        for a in range(len(this.pesos)):
            suma += this.pesos[a] * dato[a]
        if(suma > this.thresh):
            return this.clases[this.mg]
        else:
            return this.clases[int(not(this.mg))]
    def predict(this,datos):
        ret = []
        for a in datos:
            ret.append(this.pred(a))
        return ret

In [470]:
resPerceptron = correr(iris2,perceptron,holdOut)
accYMat(resPerceptron)

Arriba es la clase predicha, a la izquierda es la clase real.
Accuracy global: 1.0
<---
Elaborado sobre Iris-setosa
Accuracy: 1.0
	1	0
1	15	0
0	0	15
Elaborado sobre Iris-virginica
Accuracy: 1.0
	1	0
1	15	0
0	0	15
--->
